In [1]:
%load_ext autoreload
%load_ext autotime

time: 188 µs (started: 2024-10-08 20:01:03 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 558 µs (started: 2024-10-08 20:01:03 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 738 µs (started: 2024-10-08 20:01:03 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-resnet50-dlio-v100-node-4-v1"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/resnet50-dlio-v100-node-4-v1"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/resnet50/dlio-v100/node-4/v1/RAW/*.pfw.gz"


time: 643 µs (started: 2024-10-08 20:01:03 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 6.94 s (started: 2024-10-08 20:01:03 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 1min 36s (started: 2024-10-08 20:01:10 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          670.59 seconds                                                                                │
│  I/O Time         2.46 seconds                                                                                  │
│                   ├── Read - 2.43 seconds (99.07%)                                                              │
│                   ├── Write - 0.00 seconds (0.09%)                                                              │
│                   └── Metadata - 2.02 seconds (82.20%)                                                          │
│  I/O Operations   5,025,183 ops                                                                                 │
│                   ├── Read - 2,907,480 ops (57.86%)                                                             │
│                   ├── Write - 1,071 ops (0.02%)                                                                 │
│                   └── Metadata - 2,107,800 ops (41.94%)                                                         │
│  I/O Size         694.15 GiB                                                                                    │
│                   ├── Read - 694.15 GiB (100.00%)                                                               │
│                   └── Write - 57.67 kiB (0.00%)                                                                 │
│  Read Requests    4 kiB-256 kiB - 2,907,352 ops                                                                 │
│                   ├── <4 kiB - 43,928 ops (1.51%)                                                               │
│                   ├── 4-16 kiB - 5,088 ops (0.18%)                                                              │
│                   ├── 16-64 kiB - 7,776 ops (0.27%)                                                             │
│                   └── 64-256 kiB - 2,850,560 ops (98.05%)                                                       │
│  Write Requests   4 kiB-4 kiB - 1,071 ops                                                                       │
│                   └── <4 kiB - 1,071 ops (100.00%)                                                              │
│  Nodes            4 nodes                                                                                       │
│                   ├── 3711816138385082254 - 2.46 s (100.00%) - 173.54 GiB/10.56 kiB R/W (25.00/0.00%) -         │
│                   │   1,256,298 ops (25.00%)                                                                    │
│                   ├── 15474261681015172046 - 2.22 s (90.40%) - 173.54 GiB/10.56 kiB R/W (25.00/0.00%) -         │
│                   │   1,256,276 ops (25.00%)                                                                    │
│                   ├── 15046189470787716924 - 2.22 s (90.34%) - 173.54 GiB/10.56 kiB R/W (25.00/0.00%) -         │
│                   │   1,256,284 ops (25.00%)                                                                    │
│                   └── 15461096182755994450 - 2.17 s (88.45%) - 173.54 GiB/25.98 kiB R/W (25.00/0.00%) -         │
│                       1,256,325 ops (25.00%)                                                                    │
│  Apps             1 app                                                                                         │
│  Processes/Ranks  3,097 processes                                                                               │
│  Files            6,544 files                                                                                   │
│                   ├── Shared: 5,870 files (89.70%)                                                              │
│                   └── FPP: 674 files (10.30%)         

time: 3.51 s (started: 2024-10-08 20:02:46 -07:00)
